In [3]:
!pip install dash
!pip install jupyter-dash



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import dash 
from jupyter_dash import JupyterDash
import dash_html_components as html 
import dash_core_components as dcc
from dash.dependencies import Input, Output ,  State
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from dash import html , dcc , Dash

<ipython-input-5-27c3195c81ec>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-5-27c3195c81ec>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [6]:
data=pd.read_csv('/content/gdrive/MyDrive/IMDB Top 250 Movies.csv')
data.columns

Index(['rank', 'name', 'year', 'rating', 'genre', 'certificate', 'run_time',
       'tagline', 'budget', 'box_office', 'casts', 'directors', 'writers'],
      dtype='object')

In [7]:
genres_df = data['genre'].str.split(',', expand=True)

# Rename the columns
num_cols = len(genres_df.columns)
genres_df.columns = [f'genre_{i}' for i in range(1, num_cols+1)]

# Concatenate the new columns with the original DataFrame
data = pd.concat([data, genres_df], axis=1)

# Figures

In [8]:
imdb_colors = {
    'background': '#1F2630',
    'text': '#FFFFFF',
    'accent': '#F5C518'
}

In [9]:
df = data.copy()
# fig_1 = px.violin(df, x='certificate', y='box_office', color='genre', title='Box Office by Certificate')
fig_1 = px.violin(df, x='certificate', y='box_office', color='genre', title='Box Office by Certificate',
                  template='plotly_dark', 
                  color_discrete_sequence=px.colors.qualitative.Pastel)



In [10]:
# fig_2 = px.box(df, x='genre', y='box_office', color='genre', title='Box Office by Genre' )
fig_2 = px.box(df, x='genre', y='box_office', color='genre', title='Box Office by Genre' )


In [11]:
top_rates = data.nlargest(10, 'rating')
fig_3 = go.Figure(data=[go.Bar(x=top_rates['genre'], y=top_rates['rating'], marker={'color': '#dba506'})])
fig_3.update_layout(
plot_bgcolor=imdb_colors['background'],
paper_bgcolor=imdb_colors['background'],
font_color=imdb_colors['text']
)
# fig_3.update_layout(title='Top 10 Movies by Genre', )

In [12]:
top_movies_d = data.nlargest(10, 'rating')
fig_4 = go.Figure(data=[go.Bar(x=top_movies_d['directors'], y=top_movies_d['rating'], marker={'color': '#dba506'})])
fig_4.update_layout(
plot_bgcolor=imdb_colors['background'],
paper_bgcolor=imdb_colors['background'],
font_color=imdb_colors['text']
)
# fig_4.update_layout(title='Top 10 Movies by Directors')

In [13]:
grouped_df = df.groupby(['genre', 'year', 'rating']).size().reset_index(name='count')
fig_5 = px.sunburst(grouped_df, path=['genre', 'year', 'rating'], values='count')
fig_5.update_layout(
plot_bgcolor=imdb_colors['background'],
paper_bgcolor=imdb_colors['background'],
font_color=imdb_colors['text']
)
# fig_5.update_layout(title='Movie Ratings by Genre and Year', height = 1200, width = 1200)

In [14]:
grouped_df = data.groupby('year')['rating'].mean().reset_index()

# Create a line plot of the mean rating over time using Plotly
fig_6 = px.line(grouped_df, x='year', y='rating', title='Average Rating over Time', color_discrete_sequence=['#dba506'])
fig_6.update_layout(
plot_bgcolor=imdb_colors['background'],
paper_bgcolor=imdb_colors['background'],
font_color=imdb_colors['text']
)

# Customize the plot
# fig_6.update_layout(xaxis_title='Year', yaxis_title='Average Rating')

In [15]:
# Convert the box office column to numeric values
data['box_office'] = pd.to_numeric(data['box_office'], errors='coerce')

# Filter out missing values in the box office column
data = data.dropna(subset=['box_office'])

# Create the scatter plot
fig_7 = px.scatter(data, x='rating', y='box_office', color='genre', size='box_office', hover_name='name', 
                 title='Movie Ratings vs. Box Office Revenue')
fig_7.update_layout(
plot_bgcolor=imdb_colors['background'],
paper_bgcolor=imdb_colors['background'],
font_color=imdb_colors['text']
)

In [16]:
fig_8 = px.box(data, x='certificate', y='rating', color='certificate', 
             title='Movie Ratings by Certificate')
# fig_8.update_layout(xaxis_title='Certificate', yaxis_title='Rating')
fig_8.update_layout(
plot_bgcolor=imdb_colors['background'],
paper_bgcolor=imdb_colors['background'],
font_color=imdb_colors['text']
)

# Layout

In [17]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px

app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'], suppress_callback_exceptions=True)

# Define the options for the dropdown for graph1
# Get the unique certificate values from your data
certificates = df['certificate'].unique()

# Create a list of options for the dropdown based on the certificates
certificate_options = [{'label': cert, 'value': cert} for cert in certificates]

# Define the options for the dropdown for graph2
# Get the unique genre values from your data
genres = df['genre'].unique()

# Create a list of options for the dropdown based on the genres
genre_options = [{'label': genre, 'value': genre} for genre in genres]

# Define the IMDB color theme
imdb_colors = {
    'background': '#1F2630',
    'text': '#FFFFFF',
    'accent': '#F5C518'
}
app.layout = html.Div(children=[
    html.H1(children=[    html.Img(src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/1200px-IMDB_Logo_2016.svg.png", style={'vertical-align': 'middle', 'height': '50px'}),    "Top 250 Movies"], style={'textAlign': 'center', 'color': imdb_colors['text']})

    # html.H1("Top 250 Movies", style={'textAlign': 'center', 'color': imdb_colors['text']})
    ,
    
    html.Hr(style={'borderColor': imdb_colors['accent']}),
    
    html.Div([
        
        # Add the dropdown to graph1
        html.Div([    dcc.Graph(id="graph1", figure=fig_1),    dcc.Dropdown(id='graph1-dropdown', options=certificate_options, value=certificates[0], style={'backgroundColor': '#262626', 'color': '#000000'})
], className='six columns', style={'backgroundColor': imdb_colors['background']})





        ,

        
        
        
        # Add the dropdown to graph2
        html.Div([
            dcc.Graph(id="graph2", figure=fig_2, style={'backgroundColor': imdb_colors['background']}),
            dcc.Dropdown(id='graph2-dropdown', options=genre_options, value=genres[0] ,style={'backgroundColor': '#262626', 'color': '#000000'})
        ], className='six columns')
    ], className='row'),
    
    html.Div([
        html.Div(dcc.Graph(id="graph3", figure=fig_3, style={'backgroundColor': imdb_colors['background']}), className='six columns'),
        html.Div(dcc.Graph(id="graph4", figure=fig_4, style={'backgroundColor': imdb_colors['background']}), className='six columns')
    ], className='row', style={'marginTop': '50px'}),
    
    html.Div([
        html.Div(dcc.Graph(id="graph7", figure=fig_7, style={'backgroundColor': imdb_colors['background']}), className='six columns'),
        html.Div(dcc.Graph(id="graph6", figure=fig_6, style={'backgroundColor': imdb_colors['background']}), className='six columns')
    ], className='row', style={'marginTop': '50px'}),
    
    html.Div([
        html.Div(dcc.Graph(id="graph8", figure=fig_8, style={'backgroundColor': imdb_colors['background']}), className='eleven columns'),
    ], className='row', style={'marginTop': '50px'})
], style={'backgroundColor': imdb_colors['background'], 'color': imdb_colors['text'], 'fontFamily': 'Arial'})




# Define the custom stylesheet
app.css.append_css({
    'external_url': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/darkly/bootstrap.min.css'
})


In [18]:
fig_1.update_layout(
    plot_bgcolor=imdb_colors['background'], 
    paper_bgcolor=imdb_colors['background'],
    font_color=imdb_colors['text']
)

In [19]:
fig_1.update_layout(
    plot_bgcolor=imdb_colors['background'], 
    paper_bgcolor=imdb_colors['background'],
    font_color=imdb_colors['text']
)

In [20]:
# Define a callback to update graph1 based on the selected certificate value
@app.callback(
    dash.dependencies.Output('graph1', 'figure'),
    [dash.dependencies.Input('graph1-dropdown', 'value')])
def update_graph1(value):
    df_Certificate = df[df['certificate'] == value]
    fig = px.box(df_Certificate, x='certificate', y='box_office', color='certificate', title='Box Office by Certificate',template='plotly_dark')
    return fig


In [21]:
# Define a callback to update graph2 based on the selected genre value
@app.callback(
    dash.dependencies.Output('graph2', 'figure'),
    [dash.dependencies.Input('graph2-dropdown', 'value')])
def update_graph2(value):
    df_Genre = df[df['genre'] == value]
    fig = px.box(df_Genre, x='genre', y='box_office', color='genre', title='Box Office by Genre',template='plotly_dark')
    return fig


In [22]:
%matplotlib inline


In [23]:
app.run_server(mode="external")

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


/usr/local/lib/python3.9/dist-packages/dash/resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/darkly/bootstrap.min.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



<IPython.core.display.Javascript object>